## Intro to the FL Simulator

The [FL Simulator](https://nvflare.readthedocs.io/en/latest/user_guide/nvflare_cli/fl_simulator.html) runs a local simulation of a running NVFLARE FL deployment.  This allows researchers to test and debug an application without provisioning a real, distributed FL project. The FL Simulator runs a server and multiple clients in the same local process, with communication that mimics a real deployment.  This allows researchers to more quickly build out new components and jobs that can be directly used in a production deployment.

### Setup
The NVFlare [Getting Started Guide](https://nvflare.readthedocs.io/en/main/getting_started.html) provides instructions for setting up NVFlare on a local system or in a Docker image.  We've also cloned the NVFlare GitHub in our top-level working directory.

### Running the FL Simulator

FL Simulator usage can be displayed with the NVFlare CLI: `nvflare simulator -h`

In [ ]:
!nvflare simulator -h

For this example, we'll use the hello-numpy example, which uses NVIDIA FLARE's Recipe API.

The Recipe API provides a simplified way to define federated learning jobs in Python code rather than JSON configuration files. Examples using Recipe API include their own execution logic, so we run them directly rather than using the `nvflare simulator` command.

We'll run the hello-numpy example with 2 clients:

In [ ]:
!cd ../hello-world/hello-numpy && python job.py --n_clients 2

Watch the output above for the training to complete. The job.py script uses SimEnv (simulation environment) internally to run the federated learning job.

> **Note**: The hello-numpy example uses the Recipe API, which includes execution logic in job.py. For traditional job structures (with meta.json and app/config/ directories), you would use the `nvflare simulator` command instead.

We can check the simulation results. Recipe API jobs save results to `/tmp/nvflare/simulation/` by default:

In [ ]:
!ls -al /tmp/nvflare/simulation/hello-numpy